In [2]:
import pandas as pd
import numpy as np
import geopandas as gp
import shapely
from planet4 import io, markings

In [3]:
from coverage_utils import CoverageCalculator, get_obsids_for_region
from my_io import get_tilecoords, get_metadata, get_region_names

In [5]:
olddata = pd.read_csv("n_obj_df.csv")

In [19]:
olddata

,Latitude,Ls,MY,Which,fraction,n_objects,obsid,pixel_area,region_name
0,-85.4074,214.785,29,both,0.115,933,ESP_012079_0945,2.551911e+07,Macclesfield
1,-85.4065,183.727,29,both,0.070,1971,ESP_011407_0945,2.592302e+06,Macclesfield
2,-85.4056,198.040,29,both,0.072,1525,ESP_011723_0945,8.455380e+06,Macclesfield
3,-85.4049,207.892,29,both,0.168,788,ESP_011934_0945,7.357966e+06,Macclesfield
4,-85.4067,235.282,29,both,0.111,1285,ESP_012501_0945,4.882627e+06,Macclesfield
5,-85.4117,224.964,29,both,0.087,350,ESP_012290_0945,1.274910e+07,Macclesfield
6,-85.4086,183.683,29,both,0.074,453,ESP_011406_0945,3.767516e+06,Macclesfield
7,-85.4074,214.785,29,fans,0.088,781,ESP_012079_0945,1.942013e+07,Macclesfield
8,-85.4065,183.727,29,fans,0.046,1568,ESP_011407_0945,1.689222e+06,Macclesfield
9,-85.4056,198.040,29,fans,0.047,939,ESP_011723_0945,5.512495e+06,Macclesfield


In [7]:
data = pd.read_csv("P4_catalog_v1.1_tile_coords_final.csv")

In [31]:
data.head()

,x_tile,y_tile,x_hirise,y_hirise,PlanetocentricLatitude,PlanetographicLatitude,PositiveEast360Longitude,BodyFixedCoordinateX,BodyFixedCoordinateY,BodyFixedCoordinateZ,tile_id,obsid
0,1,1,420.0,324.0,-85.504144,-85.556701,104.952104,-68.380674,256.056147,-3370.636698,APF0000cwz,ESP_012079_0945
1,1,2,420.0,872.0,-85.502300,-85.554878,104.934805,-68.331337,256.181661,-3370.628630,APF0000ck9,ESP_012079_0945
2,1,3,420.0,1420.0,-85.500454,-85.553054,104.917486,-68.281853,256.307249,-3370.620295,APF0000cty,ESP_012079_0945
3,1,4,420.0,1968.0,-85.498607,-85.551229,104.900198,-68.232522,256.433112,-3370.615690,APF0000ciy,ESP_012079_0945
4,1,5,420.0,2516.0,-85.496762,-85.549404,104.882968,-68.183281,256.558509,-3370.606455,APF0000cwp,ESP_012079_0945


In [32]:
#x1 = data[data['x_tile'] == 1]

" Through all the coordinates for all tiles for an obsid into a Shapely object and let shapely define the boundary. This would be good to learn, b/c that’s exactly what shapely was made for and it is a standard GIS task to learn."

In [44]:
data[data['obsid'] == 'ESP_012079_0945'].x_tile.max()

5

In [22]:
allobs = data.obsid.unique() #give me every obsid

In [60]:

xmiba = []
xmaba = []
ymiba = []
ymaba = []

for obs in allobs:
    temp = data[data['obsid'] == obs] #look only at our obsid in question
    #Get xmins, xmaxes for the tiles
    xmi = temp.x_tile.min()
    xma = temp.x_tile.max()
    ymi = temp.y_tile.min()
    yma = temp.y_tile.max()
    xmiba.append(xmi)
    xmaba.append(xma)
    ymiba.append(ymi)
    ymaba.append(yma)
    
xmiba = np.asarray(xmiba)
ymiba = np.asarray(ymiba)
xmaba = np.asarray(xmaba)    
ymaba = np.asarray(ymaba)

Running and analyzing the above shows that x_tile has a min of 1, max of 5 and the y_tile has a min of 1, max of 145 across all our obsids.

In [89]:
results = 1

In [131]:
#similar to above, but now to create a dataframe which will help us get the corner values
bucket = []
for obs in allobs:
    d = {}
    temp = data[data['obsid'] == obs]
    d["x_tile_min"] = temp.x_tile.min()
    d["x_tile_max"] = temp.x_tile.max()
    d["y_tile_min"] = temp.y_tile.min()
    d["y_tile_max"] = temp.y_tile.max()
    d["obsid"] = obs
    d["top_left"] = (temp[(temp['x_tile'] == temp.x_tile.min()) & (temp["y_tile"] == temp.y_tile.min())].BodyFixedCoordinateX.values, temp[(temp['x_tile'] == temp.x_tile.min()) & (temp["y_tile"] == temp.y_tile.min())].BodyFixedCoordinateY.values)
    d["top_right"] = (temp[(temp['x_tile'] == temp.x_tile.min()) & (temp["y_tile"] == temp.y_tile.max())].BodyFixedCoordinateX.values, temp[(temp['x_tile'] == temp.x_tile.min()) & (temp["y_tile"] == temp.y_tile.max())].BodyFixedCoordinateY.values )
    d['bottom_left'] = (temp[(temp['x_tile'] == temp.x_tile.max()) & (temp["y_tile"] == temp.y_tile.min())].BodyFixedCoordinateX.values, temp[(temp['x_tile'] == temp.x_tile.max()) & (temp["y_tile"] == temp.y_tile.min())].BodyFixedCoordinateX.values)
    d['bottom_right'] = (temp[(temp['x_tile'] == temp.x_tile.max()) & (temp["y_tile"] == temp.y_tile.max())].BodyFixedCoordinateX.values, temp[(temp['x_tile'] == temp.x_tile.max()) & (temp["y_tile"] == temp.y_tile.max())].BodyFixedCoordinateY.values )
    bucket.append(d)
results = pd.DataFrame(bucket)


In [132]:
results.head() #looks weird, why are the bottom left x and bottom right x values different? curvature?  
#also what to do when there are nan values?

,bottom_left,bottom_right,obsid,top_left,top_right,x_tile_max,x_tile_min,y_tile_max,y_tile_min
0,"([-67.69881540000002], [-67.69881540000002])","([-62.388268200000006], [269.3496037])",ESP_012079_0945,"([-68.3806735], [256.0561472])","([-63.070566500000005], [269.61576610000003])",5,1,109,1
1,"([108.67046979999999], [108.67046979999999])","([121.8381508], [477.67216169999995])",ESP_011697_0980,"([108.3633138], [457.7678756])","([121.5308617], [477.87006569999994])",2,1,91,1
2,"([nan], [nan])","([nan], [nan])",ESP_021455_0935,"([nan], [nan])","([nan], [nan])",5,1,63,1
3,"([-319.50996000000004], [-319.50996000000004])","([-328.12714460000007], [-328.13045330000006])",ESP_012008_0975,"([-319.1787501], [-324.5326963])","([-327.7960465], [-328.7914651])",5,1,72,1
4,"([227.52294890000002], [227.52294890000002])","([239.92119440000002], [417.6651198])",ESP_012436_0980,"([227.279471], [406.9997462])","([239.6778595], [417.937003])",2,1,63,1


In [123]:
results.x_coord_min[0]

array([-68.3806735])

In [119]:
temp[(temp['x_tile'] == temp.x_tile.max()) & (temp["y_tile"] == temp.y_tile.max())].BodyFixedCoordinateX.values

array([221.6780802])